In [1]:
import pandas as pd
import folium

C:\Users\torsten.schmidt\AppData\Local\Temp\ipykernel_2296\1972160077.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# CSV-Daten einlesen
datei = 'globallandtemperaturesbymajorcity.csv'
df = pd.read_csv(datei)

In [3]:
# Latitude und Longitude in rein numerische Werte umwandeln
def konvertiere_breitengrad(lat):
    if 'N' in lat:
        return float(lat[:-1])
    else:
        return -float(lat[:-1])

def konvertiere_laengengrad(lon):
    if 'E' in lon:
        return float(lon[:-1])
    else:
        return -float(lon[:-1])

# Umwandlung der Längen- und Breitengrade
df['latitude'] = df['latitude'].apply(konvertiere_breitengrad)
df['longitude'] = df['longitude'].apply(konvertiere_laengengrad)

In [4]:
# Berechnung der Durchschnittstemperatur pro Stadt
df_agg = df.groupby(['city', 'latitude', 'longitude'])['averagetemperature'].mean().reset_index()

In [5]:
# Karte in Folium
karte = folium.Map(location=[0, 0], zoom_start=2)  

In [6]:
# Kreise auf Karte für jede Stadt mit r ~ T_average
for index, row in df_agg.iterrows():
    # Größe des Kreises basierend auf der Durchschnittstemperatur
    kreis_radius = (row['averagetemperature'] ) * 10000 
    folium.Circle(
        location=(row['latitude'], row['longitude']),
        radius=kreis_radius,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        popup=row['city'] + ': ' + str(round(row['averagetemperature'], 2)) + ' °C'  # Popup-Text mit Stadtname und Temperatur
    ).add_to(karte)

In [7]:
# Speichere die Karte in einer HTML-Datei
karte.save('temperaturen_weltkarte.html')

In [8]:
# Karte anzeigen
karte